### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/von_Willebrand_Disease/GSE49881'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check gene expression data availability
is_gene_available = False  # This dataset deals with microRNA, not suitable for gene expression study

# Check availability of each variable
trait_row = None
age_row = 4 if len({i.split(':')[1].strip() for i in sample_characteristics[4]}) > 1 else None
gender_row = 3 if len({i.split(':')[1].strip() for i in sample_characteristics[3]}) > 1 else None

# Define appropriate conversion functions
def convert_trait(value):
    return None  # No trait data available

def convert_age(value):
    try:
        return float(value.split(':')[1].strip())
    except:
        return None

def convert_gender(value):
    gender = value.split(':')[1].strip().lower()
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    return None

# Check if all variables are available
has_clinical_data = trait_row is not None

# Save metadata
save_cohort_info('GSE49881', './preprocessed/von_Willebrand_Disease/cohort_info.json', is_gene_available, has_clinical_data)

# Extract and save clinical feature data
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'von_Willebrand_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/von_Willebrand_Disease/trait_data/GSE49881.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
